In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# Download all three Microsoft BotBuilder Personality Chat Datasets on Google Colab virtual server
# https://drive.google.com/file/d/1TJVC6VGiAqrMzNbh_-a70FWlR6zzLuvP/view?usp=sharing
#chats.csv DOTA dataset 1.4 million
download_files = [{'id':'1TJVC6VGiAqrMzNbh_-a70FWlR6zzLuvP','file_name':'45k_before_finalise.csv'},
          {'id':'1COTg4XC0fk1cVkw1eTi1cru--O-53Cei','file_name':'50k.csv'}, 
          {'id':'1n1cIwfqqUIYipKI6Gn8tVwS9XRNNYW8q','file_name':'chat.csv'}, 
          {'id':'1O0BtTfORAGKyWKCDl6A1d5WHEgp_uIEk','file_name':'2000_henry.csv'}, 
          {'id':'1fFqaz1TdR1JxJGlLHTtsiEF7ciA4Ao7W','file_name':'match_info.csv'}, 
          ]
for entity in download_files:
  id1 = entity['id']
  downloaded = drive.CreateFile({'id':id1 }) 
  downloaded.GetContentFile(entity['file_name']) 

# 45k intent and slot top words

## for slot in intent

In [ ]:
import pandas as pd
ori_dataset = pd.read_csv('45k_before_finalise.csv')
print(ori_dataset.columns)

Index(['Unnamed: 0', 'match_id', 'conversation_id', 'utterance', 'time',
       'slot', 'unit', 'Label', 'combined_annotation', 'final_label',
       'review_label', 'word_level_data', 'slot_label',
       'combined_annotation_version2'],
      dtype='object')


In [ ]:
utter_map = {'A':{},'E':{},'I':{},'O':{}}
slot_labels = ['C','D','O','P','S','SEPA','T']
for key in utter_map.keys():
  for label in slot_labels:
    utter_map[key][label] = {}
utter_map

{'A': {'C': {}, 'D': {}, 'O': {}, 'P': {}, 'S': {}, 'SEPA': {}, 'T': {}},
 'E': {'C': {}, 'D': {}, 'O': {}, 'P': {}, 'S': {}, 'SEPA': {}, 'T': {}},
 'I': {'C': {}, 'D': {}, 'O': {}, 'P': {}, 'S': {}, 'SEPA': {}, 'T': {}},
 'O': {'C': {}, 'D': {}, 'O': {}, 'P': {}, 'S': {}, 'SEPA': {}, 'T': {}}}

In [ ]:
texts = ori_dataset['word_level_data'].tolist()
slot_labels = ori_dataset['slot_label'].tolist()
intent_labels = ori_dataset['final_label'].tolist()
assert len(texts) == len(slot_labels)
for i in range(len(texts)):
  if str(texts[i]) == 'nan':
    continue
  text_arr = str(texts[i]).split()
  slot_label_arr = slot_labels[i].split()
  assert len(text_arr) == len(slot_label_arr)
  intent_label = intent_labels[i]
  assert intent_label in utter_map
  slot_map = utter_map[intent_label]
  for j in range(len(text_arr)):
    label = str(slot_label_arr[j]).strip() 
    assert label in slot_map
    inner_map = slot_map[label]
    text = str(text_arr[j]).lower().strip()
    if text in inner_map:
      inner_map[text]+=1
    else:
      inner_map[text]=1
utter_map

In [ ]:
def get_top_words_for_each_slot_label_in_intent(intent_label, slot_label ,whole_map):
  slot_map = whole_map[intent_label]
  each_map = slot_map[slot_label]
  each_map = sorted(each_map.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)
  print(each_map)
get_top_words_for_each_slot_label_in_intent('A','C',utter_map)

[('sf', 73), ('pudge', 41), ('am', 37), ('wr', 34), ('invoker', 27), ('pa', 26), ('slark', 24), ('es', 24), ('qop', 23), ('ta', 22), ('doom', 21), ('tiny', 18), ('void', 17), ('tusk', 17), ('sniper', 17), ('ember', 17), ('aa', 17), ('axe', 16), ('zeus', 15), ('silencer', 15), ('rubick', 15), ('lion', 15), ('slardar', 14), ('lina', 14), ('jugg', 14), ('alche', 14), ('alch', 14), ('sk', 12), ('riki', 11), ('lich', 11), ('gyro', 11), ('clinkz', 11), ('mirana', 10), ('invo', 10), ('clock', 10), ('wind', 9), ('sven', 9), ('omni', 9), ('legion', 9), ('dazzle', 9), ('puck', 8), ('necro', 8), ('medusa', 8), ('jug', 8), ('drow', 8), ('bh', 8), ('troll', 7), ('storm', 7), ('morph', 7), ('magnus', 7), ('huskar', 7), ('enigma', 7), ('ursa', 6), ('tinker', 6), ('pl', 6), ('nyx', 6), ('meepo', 6), ('luna', 6), ('cm', 6), ('bara', 6), ('ww', 5), ('wd', 5), ('venge', 5), ('terror', 5), ('sky', 5), ('shaker', 5), ('sb', 5), ('pugna', 5), ('potm', 5), ('naga', 5), ('kotl', 5), ('dk', 5), ('brood', 5), (

In [ ]:
get_top_words_for_each_slot_label_in_intent('A','D',utter_map)

[('dota', 12), ('feed', 9), ('lc', 7), ('mmr', 6), ('invoke', 6), ('venom', 4), ('spirit', 3), ('force', 3), ('aghs', 3), ('yasha', 2), ('tango', 1), ('shackleshot', 1), ('sange', 1), ('rage', 1), ('quas', 1), ('pounce', 1), ('phase', 1), ('mom', 1), ('mkb', 1), ('dagon', 1), ('couriers', 1), ('courier', 1), ('control', 1), ('burro', 1), ('boots', 1), ('blink', 1), ('bkb', 1)]


In [ ]:
get_top_words_for_each_slot_label_in_intent('A','O',utter_map)

[('commend', 344), ('pls', 270), ('g', 239), ('wait', 229), ('w8', 179), ('for', 141), ('and', 139), ('please', 136), ('pause', 135), ('ty', 119), ('plz', 96), ('reported', 91), ('to', 80), ('is', 74), ('team', 71), ('just', 68), ('can', 64), ('game', 60), ('no', 57), ('will', 53), ('min', 46), ('lol', 43), ('if', 43), ('dc', 43), ('end', 41), ('a', 41), ('the', 40), ('comend', 40), (':d', 40), ('dont', 38), ('1', 35), ('unpause', 34), ('2', 34), ('not', 33), ('so', 32), ('ok', 32), (':)', 29), ('in', 28), ('on', 27), ('commended', 25), ('have', 24), ('wk', 23), ('now', 23), ('im', 23), ('want', 22), ('thx', 21), ('come', 21), ('cant', 21), ('be', 21), ('nice', 20), ('need', 20), ('with', 19), ('sec', 18), ('reprot', 18), ('let', 18), ('do', 18), ('thanks', 17), ('s', 17), ('or', 17), ('hes', 17), ('haha', 17), ('but', 17), ('4', 16), ('why', 15), ('too', 15), ('of', 15), ('good', 15), ('finish', 15), ('3', 15), ('time', 14), ('said', 14), ('more', 14), ('mins', 14), ('leave', 14), ('g

In [ ]:
get_top_words_for_each_slot_label_in_intent('A','P',utter_map)

[('me', 220), ('i', 172), ('u', 129), ('he', 111), ('him', 98), ('you', 96), ('we', 78), ('this', 72), ('all', 66), ('guys', 58), ('my', 56), ('it', 41), ('that', 21), ('us', 20), ('ur', 19), ('what', 18), ('your', 15), ('our', 14), ('them', 13), ('its', 13), ('who', 11), ('they', 11), ('dude', 9), ('bro', 8), ('man', 7), ('she', 5), ('one', 5), ('his', 5), ('nobody', 4), ('guy', 4), ('as', 4), ('thats', 3), ('everyone', 3), ('ye', 2), ('there', 2), ('their', 2), ('something', 2), ('some', 2), ('many', 2), ('girl', 2), ('few', 2), ('anything', 2), ('any', 2), ('where', 1), ('whatever', 1), ('those', 1), ('these', 1), ('such', 1), ('someone', 1), ('nothing', 1), ('enough', 1), ('each', 1), ('bros', 1), ('both', 1), ('anybody', 1), ('another', 1)]


In [ ]:
get_top_words_for_each_slot_label_in_intent('A','S',utter_map)

[('report', 992), ('afk', 184), ('gg', 134), ('go', 57), ('wp', 37), ('mid', 29), ('back', 27), ('ggwp', 21), ('ez', 21), ('ff', 15), ('push', 13), ('help', 13), ('support', 10), ('feeding', 10), ('carry', 10), ('farm', 8), ('suport', 6), ('reports', 6), ('lane', 6), ('top', 5), ('skill', 5), ('rosh', 5), ('reporta', 5), ('repor', 5), ('offlane', 5), ('feeder', 5), ('bs', 5), ('blood', 5), ('zz', 4), ('stay', 4), ('solo', 4), ('safe', 4), ('rat', 4), ('gj', 4), ('care', 4), ('base', 4), ('war', 3), ('ulti', 3), ('ult', 3), ('take', 3), ('sup', 3), ('stun', 3), ('rofl', 3), ('miss', 3), ('ks', 3), ('jungle', 3), ('damage', 3), ('bot', 3), ('throw', 2), ('throne', 2), ('supports', 2), ('skillz', 2), ('roshan', 2), ('gank', 2), ('farming', 2), ('dive', 2), ('cw', 2), ('creeps', 2), ('cancer', 2), ('block', 2), ('wards', 1), ('tp', 1), ('tower', 1), ('reporto', 1), ('rax', 1), ('pushed', 1), ('off', 1), ('lanes', 1), ('invis', 1), ('hex', 1), ('gl', 1), ('fountain', 1), ('cooldown', 1), ('

In [ ]:
get_top_words_for_each_slot_label_in_intent('A','T',utter_map)

[('wtf', 16), ('fucking', 11), ('abuse', 10), ('noob', 8), ('shit', 4), ('scum', 4), ('dead', 4), ('retard', 3), ('tards', 2), ('pussy', 2), ('lmfao', 2), ('fuck', 2), ('fu', 2), ('ffs', 2), ('fat', 2), ('black', 2), ('bitch', 2), ('wasted', 1), ('tard', 1), ('stupids', 1), ('stupid', 1), ('stfu', 1), ('rube', 1), ('retards', 1), ('peru', 1), ('pee', 1), ('morons', 1), ('moron', 1), ('mofo', 1), ('idiota', 1), ('hole', 1), ('gay', 1), ('fuk', 1), ('fuckign', 1), ('fucker', 1), ('fking', 1), ('dildo', 1), ('dickhead', 1), ('cumming', 1), ('cum', 1), ('cok', 1), ('beast', 1), ('balls', 1), ('backdoor', 1), ('anus', 1)]


In [ ]:
get_top_words_for_each_slot_label_in_intent('E','C',utter_map)

[('am', 111), ('sf', 106), ('qop', 63), ('wr', 59), ('pudge', 54), ('mother', 45), ('invoker', 44), ('ta', 43), ('alche', 38), ('riki', 36), ('pa', 36), ('es', 33), ('lion', 32), ('doom', 32), ('slark', 31), ('ember', 31), ('tiny', 29), ('sniper', 28), ('axe', 28), ('lina', 27), ('alch', 27), ('tusk', 24), ('silencer', 24), ('void', 23), ('rubick', 23), ('jugg', 23), ('sb', 21), ('cm', 20), ('necro', 19), ('mirana', 19), ('bh', 19), ('slardar', 17), ('nyx', 15), ('aa', 15), ('viper', 14), ('ursa', 14), ('storm', 14), ('omni', 14), ('venge', 13), ('gyro', 13), ('bara', 13), ('alchemist', 13), ('pl', 12), ('magnus', 12), ('legion', 12), ('clock', 12), ('zeus', 11), ('wind', 11), ('ss', 11), ('sky', 11), ('puck', 11), ('meepo', 11), ('luna', 11), ('lich', 11), ('dazzle', 11), ('wd', 10), ('tinker', 10), ('sk', 10), ('od', 10), ('bb', 10), ('weaver', 9), ('sven', 9), ('jug', 9), ('io', 9), ('invo', 9), ('huskar', 9), ('brood', 9), ('alchi', 9), ('ww', 8), ('veno', 8), ('undying', 8), ('tro

In [ ]:
get_top_words_for_each_slot_label_in_intent('E','D',utter_map)

[('mmr', 56), ('mom', 46), ('dota', 42), ('feed', 33), ('lc', 30), ('dagon', 9), ('rage', 8), ('head', 8), ('gem', 7), ('midas', 6), ('blink', 6), ('bkb', 6), ('dagger', 5), ('courier', 5), ('boots', 5), ('dust', 4), ('basher', 4), ('shield', 3), ('shackle', 3), ('refraction', 3), ('manta', 3), ('flying', 3), ('duel', 3), ('crit', 3), ('control', 3), ('bottle', 3), ('agh', 3), ('aegis', 3), ('wand', 2), ('venom', 2), ('treads', 2), ('track', 2), ('spirit', 2), ('smoke', 2), ('silence', 2), ('rot', 2), ('radiance', 2), ('powershot', 2), ('orchid', 2), ('leap', 2), ('invoke', 2), ('heart', 2), ('donkey', 2), ('blademail', 2), ('aghs', 2), ('yasha', 1), ('vlads', 1), ('trap', 1), ('toss', 1), ('tornado', 1), ('tombstone', 1), ('tangos', 1), ('soul', 1), ('snowball', 1), ('skewer', 1), ('skadi', 1), ('shadowblade', 1), ('shackleshot', 1), ('shackles', 1), ('shackled', 1), ('sentrys', 1), ('sentry', 1), ('sentries', 1), ('sange', 1), ('ruptured', 1), ('ring', 1), ('relic', 1), ('refresher',

In [ ]:
get_top_words_for_each_slot_label_in_intent('E','O',utter_map)

[('is', 383), ('a', 366), ('team', 360), ('and', 328), ('the', 312), ('so', 281), ('to', 228), ('game', 227), ('are', 200), ('lol', 181), ('no', 174), ('for', 166), ('trash', 165), ('just', 153), ('of', 150), ('in', 149), ('like', 137), ('dont', 131), ('pls', 127), ('up', 113), ('with', 110), ('have', 106), ('on', 105), ('not', 105), ('can', 98), ('why', 90), ('im', 85), ('win', 83), ('nice', 83), ('do', 80), ('get', 78), ('shut', 76), ('end', 74), ('how', 72), ('bad', 72), ('too', 71), ('play', 71), ('was', 70), ('but', 69), ('die', 66), (':d', 66), ('haha', 65), ('yeah', 57), ('got', 56), ('1', 56), ('at', 55), ('good', 54), ('pick', 52), ('now', 52), ('cant', 52), ('kill', 51), ('wait', 50), ('still', 50), ('russian', 50), ('please', 50), ('2', 50), ('will', 49), ('if', 49), ('useless', 48), ('more', 48), ('see', 47), ('peruvian', 47), ('talk', 46), ('plz', 46), ('stop', 43), ('3', 41), ('time', 40), ('say', 40), ('know', 40), ('4k', 40), ('4', 40), ('worst', 39), ('player', 39), ('

In [ ]:
get_top_words_for_each_slot_label_in_intent('E','P',utter_map)

[('u', 710), ('you', 621), ('i', 471), ('this', 393), ('my', 266), ('me', 206), ('that', 189), ('he', 179), ('your', 160), ('ur', 159), ('all', 142), ('what', 129), ('we', 102), ('it', 97), ('him', 74), ('guys', 68), ('they', 55), ('our', 45), ('its', 45), ('as', 45), ('one', 38), ('who', 33), ('dude', 33), ('his', 31), ('thats', 28), ('man', 27), ('these', 24), ('such', 23), ('some', 23), ('them', 22), ('nothing', 19), ('us', 18), ('cuz', 18), ('yourself', 14), ('most', 14), ('where', 13), ('there', 13), ('any', 13), ('bro', 11), ('another', 11), ('her', 10), ('she', 9), ('everyone', 9), ('both', 9), ('something', 8), ('guy', 8), ('enough', 8), ('whats', 7), ('many', 7), ('ye', 6), ('others', 6), ('other', 6), ('those', 5), ('their', 5), ('someone', 5), ('anything', 5), ('urself', 4), ('theres', 4), ('some1', 4), ('girl', 4), ('everything', 4), ('wheres', 3), ('whatever', 3), ('sister', 3), ('men', 3), ('yourselves', 2), ('which', 2), ('theyre', 2), ('nobody', 2), ('mine', 2), ('girls

In [ ]:
get_top_words_for_each_slot_label_in_intent('E','S',utter_map)

[('gg', 239), ('report', 237), ('ez', 191), ('mid', 169), ('go', 114), ('off', 94), ('carry', 79), ('wp', 65), ('back', 36), ('rofl', 34), ('farm', 33), ('ggwp', 28), ('lane', 27), ('support', 22), ('wards', 20), ('afk', 20), ('top', 19), ('ff', 17), ('bot', 17), ('take', 16), ('cancer', 16), ('feeding', 14), ('ulti', 13), ('help', 13), ('bs', 13), ('stun', 12), ('skill', 12), ('rat', 12), ('jungle', 12), ('b', 12), ('supports', 11), ('solo', 11), ('push', 11), ('gank', 11), ('care', 11), ('tp', 10), ('gl', 10), ('ward', 9), ('throw', 9), ('stay', 9), ('feeder', 8), ('farming', 8), ('creeps', 8), ('blood', 8), ('ult', 7), ('lanes', 7), ('creep', 7), ('reports', 6), ('mana', 6), ('base', 6), ('tower', 5), ('miss', 5), ('gj', 5), ('cares', 5), ('trilane', 4), ('sup', 4), ('rosh', 4), ('rax', 4), ('offlane', 4), ('invis', 4), ('ganks', 4), ('fb', 4), ('carrys', 4), ('aggro', 4), ('safelane', 3), ('ks', 3), ('fountain', 3), ('eziest', 3), ('exp', 3), ('damage', 3), ('chicken', 3), ('bottom

In [ ]:
get_top_words_for_each_slot_label_in_intent('E','T',utter_map)

[('noob', 878), ('fuck', 807), ('fucking', 593), ('shit', 546), ('idiot', 222), ('retard', 148), ('bitch', 139), ('stupid', 132), ('suck', 126), ('noobs', 120), ('stfu', 110), ('wtf', 96), ('gay', 91), ('dick', 81), ('cunt', 77), ('retarded', 73), ('ass', 71), ('idiots', 59), ('fucker', 55), ('peru', 53), ('faggot', 50), ('dumb', 48), ('retards', 43), ('nigga', 43), ('fuckin', 43), ('pussy', 42), ('sucks', 39), ('fucked', 39), ('rape', 33), ('fu', 31), ('bitches', 31), ('shitty', 30), ('moron', 25), ('cunts', 24), ('fucks', 23), ('fck', 22), ('asshole', 22), ('fking', 21), ('dead', 19), ('raped', 18), ('fag', 18), ('peruvians', 17), ('fuckign', 17), ('bastard', 17), ('fuk', 16), ('cock', 16), ('fuckers', 15), ('fcking', 14), ('nigger', 13), ('bullshit', 13), ('tard', 12), ('shits', 12), ('noobest', 10), ('morons', 10), ('fucktard', 10), ('faggots', 10), ('dogshit', 10), ('pussies', 9), ('piss', 9), ('motherfucker', 9), ('hook', 9), ('gays', 9), ('whore', 8), ('porn', 8), ('fucken', 8),

In [ ]:
get_top_words_for_each_slot_label_in_intent('I','C',utter_map)

[('am', 33), ('sf', 29), ('alche', 27), ('pudge', 25), ('slark', 23), ('wr', 19), ('qop', 19), ('invoker', 19), ('pa', 18), ('es', 17), ('doom', 17), ('void', 13), ('ta', 13), ('alch', 12), ('tiny', 9), ('bh', 9), ('slardar', 8), ('mirana', 8), ('invo', 8), ('bara', 8), ('puck', 7), ('od', 7), ('necro', 7), ('meepo', 7), ('lion', 7), ('cm', 7), ('lina', 6), ('ember', 6), ('dazzle', 6), ('tinker', 5), ('storm', 5), ('sniper', 5), ('silencer', 5), ('omni', 5), ('lich', 5), ('enigma', 5), ('drow', 5), ('dk', 5), ('weaver', 4), ('sven', 4), ('rubick', 4), ('riki', 4), ('jugg', 4), ('jug', 4), ('clock', 4), ('alchemist', 4), ('aa', 4), ('zeus', 3), ('voker', 3), ('ursa', 3), ('ss', 3), ('sky', 3), ('sk', 3), ('pl', 3), ('deaths', 3), ('clinkz', 3), ('axe', 3), ('wyvern', 2), ('wind', 2), ('viper', 2), ('tusk', 2), ('spectre', 2), ('slayer', 2), ('sd', 2), ('sb', 2), ('oracle', 2), ('ogre', 2), ('naga', 2), ('magnus', 2), ('jugger', 2), ('io', 2), ('furion', 2), ('dusa', 2), ('dp', 2), ('bb'

In [ ]:
get_top_words_for_each_slot_label_in_intent('I','D',utter_map)

[('mmr', 90), ('feed', 46), ('rage', 26), ('dota', 20), ('lc', 7), ('duel', 4), ('phase', 3), ('midas', 3), ('dagon', 3), ('spirit', 2), ('sentry', 2), ('league', 2), ('couriers', 2), ('boots', 2), ('blink', 2), ('bkb', 2), ('wex', 1), ('vlads', 1), ('vacum', 1), ('urn', 1), ('track', 1), ('shrapnel', 1), ('sentrys', 1), ('rapier', 1), ('radi', 1), ('pounce', 1), ('orchid', 1), ('heart', 1), ('gem', 1), ('euls', 1), ('eul', 1), ('enchant', 1), ('dotabuff', 1), ('divine', 1), ('courier', 1), ('bottles', 1), ('bottle', 1), ('blademail', 1), ('blade', 1), ('armlet', 1), ('agha', 1), ('aegis', 1), ('25mmr', 1)]


In [ ]:
get_top_words_for_each_slot_label_in_intent('I','O',utter_map)

[('game', 201), ('so', 106), ('is', 90), ('lol', 84), ('to', 72), ('and', 70), ('for', 66), ('easy', 66), ('a', 58), ('life', 53), ('team', 50), ('in', 48), ('1', 45), ('too', 44), ('0', 43), ('haha', 41), ('4', 41), ('of', 40), ('2', 40), ('no', 38), ('kill', 37), ('the', 36), ('mad', 36), ('just', 36), ('win', 35), ('are', 33), (':d', 33), ('can', 31), ('pls', 30), ('not', 30), ('kid', 30), ('3', 30), ('why', 28), ('quit', 28), ('5', 28), ('nice', 26), ('fail', 26), ('account', 26), ('with', 25), ('lose', 24), ('play', 23), ('like', 22), ('have', 22), ('buyer', 22), ('bad', 22), ('at', 22), ('2k', 22), ('pick', 21), ('hard', 21), ('dont', 21), ('4k', 21), ('still', 20), ('sad', 20), ('how', 20), ('1k', 20), ('izi', 19), ('was', 18), ('on', 18), ('im', 18), ('hahaha', 18), ('blame', 18), ('10', 18), ('more', 17), ('easiest', 17), ('9', 17), ('when', 16), ('vs', 16), ('plz', 16), ('kids', 16), ('end', 16), ('be', 16), ('8', 16), ('7', 16), ('13', 16), ('12', 16), ('11', 16), ('will', 1

In [ ]:
get_top_words_for_each_slot_label_in_intent('I','P',utter_map)

[('u', 136), ('i', 103), ('you', 84), ('this', 78), ('me', 65), ('my', 60), ('he', 41), ('it', 34), ('all', 30), ('that', 29), ('your', 25), ('ur', 25), ('we', 23), ('its', 21), ('what', 17), ('they', 16), ('guys', 16), ('him', 14), ('bro', 12), ('his', 11), ('as', 11), ('us', 8), ('thats', 7), ('our', 7), ('who', 6), ('them', 6), ('nothing', 5), ('dude', 5), ('there', 4), ('such', 4), ('someone', 4), ('some', 4), ('one', 4), ('man', 4), ('guy', 4), ('she', 3), ('most', 3), ('ye', 2), ('where', 2), ('thy', 2), ('those', 2), ('these', 2), ('their', 2), ('something', 2), ('others', 2), ('men', 2), ('itself', 2), ('cuz', 2), ('anyone', 2), ('any', 2), ('another', 2), ('yall', 1), ('wheres', 1), ('urself', 1), ('theyre', 1), ('sister', 1), ('many', 1), ('her', 1), ('guyz', 1), ('girls', 1), ('girl', 1), ('few', 1), ('everything', 1), ('everyone', 1), ('each', 1), ('dudes', 1), ('both', 1), ('anything', 1)]


In [ ]:
get_top_words_for_each_slot_label_in_intent('I','S',utter_map)

[('ez', 1932), ('mid', 287), ('gg', 169), ('report', 47), ('go', 38), ('top', 36), ('wp', 35), ('bot', 27), ('carry', 26), ('feeding', 23), ('farm', 17), ('rofl', 12), ('ggwp', 11), ('feeder', 11), ('bait', 11), ('back', 11), ('wards', 10), ('lane', 10), ('take', 9), ('support', 9), ('ff', 9), ('ezzz', 9), ('afk', 9), ('offlane', 8), ('off', 7), ('fb', 7), ('solo', 6), ('help', 6), ('ezz', 6), ('cancer', 6), ('jungle', 5), ('bs', 5), ('zz', 4), ('sup', 4), ('push', 4), ('lanes', 4), ('ks', 4), ('gank', 4), ('farming', 4), ('eziest', 4), ('creeps', 4), ('chicken', 4), ('ward', 3), ('tp', 3), ('safe', 3), ('rune', 3), ('rat', 3), ('mana', 3), ('carrys', 3), ('ulti', 2), ('ult', 2), ('tower', 2), ('throw', 2), ('supports', 2), ('skill', 2), ('reports', 2), ('rax', 2), ('miss', 2), ('micro', 2), ('hardlane', 2), ('gj', 2), ('fountain', 2), ('ezzzzzz', 2), ('ezzzz', 2), ('ezy', 2), ('ezwp', 2), ('ezmid', 2), ('ezi', 2), ('dive', 2), ('damage', 2), ('care', 2), ('btm', 2), ('bottom', 2), ('b

In [ ]:
get_top_words_for_each_slot_label_in_intent('I','T',utter_map)

[('wtf', 32), ('fucking', 11), ('dead', 9), ('hook', 6), ('fuck', 5), ('noob', 4), ('abuse', 4), ('shit', 3), ('retarded', 3), ('ffs', 3), ('retard', 2), ('rape', 2), ('moron', 2), ('fking', 2), ('boom', 2), ('bitches', 2), ('ass', 2), ('tramp', 1), ('tards', 1), ('stupids', 1), ('stupid', 1), ('sexy', 1), ('sadist', 1), ('russki', 1), ('retards', 1), ('raped', 1), ('pussy', 1), ('peruvians', 1), ('noobs', 1), ('mams', 1), ('looser', 1), ('laid', 1), ('israel', 1), ('idiots', 1), ('gay', 1), ('fucked', 1), ('fu', 1), ('faggot', 1), ('enemy', 1), ('cunt', 1), ('cum', 1), ('crackwhores', 1), ('bomb', 1), ('blackout', 1), ('bitch', 1)]


In [ ]:
get_top_words_for_each_slot_label_in_intent('O','C',utter_map)

[('am', 309), ('sf', 240), ('wr', 163), ('doom', 147), ('pudge', 130), ('es', 124), ('qop', 112), ('pa', 100), ('slark', 99), ('invoker', 91), ('alche', 71), ('riki', 70), ('ta', 66), ('cm', 66), ('tiny', 64), ('lina', 64), ('alch', 64), ('bb', 63), ('slardar', 61), ('lion', 61), ('mirana', 60), ('ember', 56), ('aa', 55), ('sniper', 53), ('jugg', 49), ('bh', 48), ('void', 47), ('meepo', 45), ('silencer', 44), ('sk', 43), ('invo', 43), ('zeus', 41), ('sven', 40), ('axe', 39), ('sb', 38), ('pl', 37), ('necro', 36), ('huskar', 36), ('storm', 35), ('viper', 34), ('rubick', 34), ('ss', 32), ('spectre', 32), ('omni', 32), ('dazzle', 31), ('ursa', 30), ('tinker', 29), ('jug', 28), ('ww', 27), ('gyro', 26), ('wd', 25), ('magnus', 25), ('legion', 25), ('enigma', 25), ('luna', 24), ('ogre', 23), ('clock', 23), ('bara', 22), ('troll', 21), ('tide', 21), ('lich', 21), ('tusk', 20), ('techies', 20), ('drow', 20), ('brood', 20), ('alchemist', 20), ('shaker', 19), ('veno', 18), ('puck', 18), ('medusa

In [ ]:
get_top_words_for_each_slot_label_in_intent('O','D',utter_map)

[('mmr', 156), ('dota', 131), ('lc', 75), ('feed', 71), ('bkb', 47), ('blink', 45), ('courier', 34), ('rage', 31), ('gem', 29), ('duel', 29), ('smoke', 23), ('dust', 23), ('hood', 21), ('boots', 21), ('aegis', 21), ('manta', 18), ('dagger', 18), ('crit', 18), ('rapier', 17), ('midas', 17), ('aghs', 14), ('silence', 13), ('sentry', 13), ('mkb', 13), ('heart', 13), ('dagon', 12), ('radiance', 11), ('divine', 11), ('shackle', 10), ('mek', 10), ('mom', 9), ('euls', 9), ('trap', 8), ('head', 8), ('force', 8), ('bottle', 8), ('spirit', 7), ('shackles', 7), ('meka', 7), ('blademail', 7), ('stick', 6), ('sentries', 6), ('phase', 6), ('linkens', 6), ('league', 6), ('deso', 6), ('agha', 6), ('track', 5), ('refresher', 5), ('pipe', 5), ('ghost', 5), ('couriers', 5), ('butterfly', 5), ('armlet', 5), ('yasha', 4), ('vlad', 4), ('tango', 4), ('sentrys', 4), ('satanic', 4), ('ravage', 4), ('maphack', 4), ('invoke', 4), ('glimpse', 4), ('flying', 4), ('eul', 4), ('dota2', 4), ('diffusal', 4), ('daedal

In [ ]:
get_top_words_for_each_slot_label_in_intent('O','O',utter_map)

[('lol', 2106), ('is', 1004), ('haha', 899), ('the', 857), ('a', 831), ('to', 781), ('no', 733), ('game', 714), ('for', 615), ('so', 608), ('not', 596), ('and', 590), ('nice', 563), (':d', 563), ('end', 535), ('xd', 496), ('ty', 487), ('just', 444), ('dont', 443), ('in', 416), ('hahaha', 402), ('have', 399), ('why', 379), ('good', 369), ('are', 346), ('team', 338), ('im', 331), ('ok', 330), ('was', 308), ('like', 306), ('can', 306), ('of', 302), ('how', 297), ('on', 295), ('pls', 278), ('win', 275), ('do', 265), (':)', 256), ('got', 252), ('2', 248), ('1', 240), ('wait', 237), ('time', 221), ('with', 217), ('yeah', 214), ('did', 213), ('get', 211), ('too', 210), ('more', 205), ('now', 203), ('kill', 202), ('pro', 201), ('wow', 199), ('but', 196), ('cant', 193), ('come', 192), ('know', 191), ('if', 189), ('play', 188), ('be', 187), ('sad', 186), ('will', 185), ('well', 182), (':(', 179), ('really', 173), ('please', 172), ('bad', 172), ('at', 171), ('see', 170), ('lmao', 164), (':', 164)

In [ ]:
get_top_words_for_each_slot_label_in_intent('O','P',utter_map)

[('i', 1737), ('u', 1212), ('you', 1207), ('me', 818), ('he', 690), ('that', 575), ('it', 565), ('this', 549), ('we', 542), ('my', 453), ('what', 385), ('guys', 342), ('all', 298), ('its', 285), ('ur', 265), ('your', 264), ('him', 163), ('one', 161), ('they', 142), ('man', 138), ('who', 132), ('bro', 125), ('us', 113), ('dude', 113), ('there', 106), ('thats', 102), ('his', 101), ('as', 89), ('where', 83), ('our', 83), ('them', 66), ('some', 55), ('nothing', 44), ('many', 40), ('ye', 36), ('any', 36), ('enough', 33), ('cuz', 33), ('whats', 31), ('she', 31), ('other', 29), ('guy', 29), ('these', 28), ('something', 28), ('everyone', 28), ('such', 23), ('anyone', 22), ('someone', 21), ('whatever', 20), ('anything', 19), ('those', 18), ('mine', 17), ('her', 17), ('their', 16), ('men', 16), ('girl', 16), ('another', 15), ('either', 13), ('most', 12), ('wheres', 11), ('theres', 11), ('girls', 11), ('everything', 11), ('which', 10), ('yall', 9), ('each', 9), ('both', 8), ('yourself', 7), ('you

In [ ]:
get_top_words_for_each_slot_label_in_intent('O','S',utter_map)

[('gg', 3735), ('wp', 1115), ('ggwp', 776), ('mid', 413), ('go', 383), ('rofl', 234), ('back', 179), ('carry', 154), ('gl', 141), ('farm', 124), ('ff', 114), ('top', 109), ('support', 88), ('lane', 88), ('ult', 70), ('bot', 70), ('ulti', 67), ('take', 64), ('gj', 60), ('solo', 59), ('glhf', 58), ('b', 55), ('help', 54), ('wards', 51), ('tp', 49), ('skill', 46), ('push', 45), ('stun', 44), ('creeps', 44), ('miss', 42), ('care', 42), ('zz', 41), ('ward', 41), ('jungle', 41), ('rat', 39), ('rosh', 38), ('throw', 37), ('tower', 36), ('sup', 36), ('off', 36), ('mana', 33), ('gank', 33), ('bs', 32), ('damage', 31), ('throne', 26), ('ez', 26), ('creep', 26), ('base', 26), ('feeding', 25), ('supports', 24), ('safe', 24), ('farming', 24), ('blood', 24), ('tank', 21), ('offlane', 20), ('bait', 20), ('stay', 17), ('fb', 17), ('cares', 17), ('rune', 16), ('report', 16), ('cancer', 16), ('rp', 15), ('cd', 15), ('ks', 14), ('fountain', 14), ('dive', 14), ('deny', 14), ('afk', 14), ('heal', 13), ('gg

In [ ]:
get_top_words_for_each_slot_label_in_intent('O','T',utter_map)

[('wtf', 331), ('dead', 89), ('fuck', 84), ('hook', 69), ('shit', 39), ('lmfao', 38), ('fat', 35), ('fucking', 31), ('noob', 28), ('ffs', 22), ('boom', 22), ('dumb', 12), ('sexy', 11), ('enemy', 11), ('dafuq', 11), ('kkk', 10), ('tard', 9), ('smack', 9), ('cum', 9), ('black', 9), ('whore', 8), ('hole', 8), ('dick', 8), ('stupid', 7), ('retards', 7), ('pee', 7), ('fcking', 7), ('fck', 7), ('crap', 7), ('ass', 7), ('raped', 6), ('fu', 6), ('wasted', 5), ('suck', 5), ('slut', 5), ('pud', 5), ('israel', 5), ('idiot', 5), ('bra', 5), ('beast', 5), ('autistic', 5), ('autism', 5), ('tards', 4), ('sucks', 4), ('stfu', 4), ('sex', 4), ('gays', 4), ('gay', 4), ('fucked', 4), ('dildo', 4), ('cunts', 4), ('balls', 4), ('whit', 3), ('tits', 3), ('tang', 3), ('scum', 3), ('noone', 3), ('mofo', 3), ('idiots', 3), ('fucks', 3), ('drug', 3), ('cumming', 3), ('cummin', 3), ('cuming', 3), ('cocky', 3), ('bonert', 3), ('bomb', 3), ('blow', 3), ('bi', 3), ('abuse', 3), ('snatch', 2), ('sluts', 2), ('shitty